In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import sys, os
from random import shuffle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from utils import *
import datetime
import matplotlib.pyplot as plt
import pickle
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
import gc


In [3]:
def return_omics_type(data):
    t = 0
    temp = []
    name = "_"
    for k, v in data.items():
        t = t + v
        if(v):
            temp.append(k)
    temp = tuple(temp)
    if t == 1:
        return "1omics", name.join(temp)
    if t == 2:
        return "2omics", name.join(temp)
    if t == 3:
        return "3omics", name.join(temp)
        
data_types = [
    {"ge":1, "mut":1, "meth":1},
    {"ge":1, "mut":1, "meth":0},
    {"ge":1, "mut":0, "meth":1},
    {"ge":0, "mut":1, "meth":1},
    {"ge":1, "mut":0, "meth":0},
    {"ge":0, "mut":1, "meth":0},
    {"ge":0, "mut":0, "meth":1},    
]
data_sets = ["all_test"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define model

In [4]:
class GINConvNet(torch.nn.Module):
    def __init__(self, n_output=1,num_features_xd=78, num_features_xt=25,
                n_filters=32, embed_dim=128, output_dim=128, dropout=0.2,
                out_tissue_d=13, ge=0, mut=0, meth=0):

        super(GINConvNet, self).__init__()
        self.ge = ge
        self.mut = mut
        self.meth = meth
        print(self.ge, self.mut, self.meth)

        dim = 32
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.n_output = n_output
        # convolution layers
        nn1 = Sequential(Linear(num_features_xd, dim), ReLU(), Linear(dim, dim))
        self.conv1 = GINConv(nn1)
        self.bn1 = torch.nn.BatchNorm1d(dim)

        nn2 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv2 = GINConv(nn2)
        self.bn2 = torch.nn.BatchNorm1d(dim)

        nn3 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv3 = GINConv(nn3)
        self.bn3 = torch.nn.BatchNorm1d(dim)

        nn4 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv4 = GINConv(nn4)
        self.bn4 = torch.nn.BatchNorm1d(dim)

        nn5 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv5 = GINConv(nn5)
        self.bn5 = torch.nn.BatchNorm1d(dim)

        self.fc1_xd = Linear(dim, output_dim)

        # cell line feature
        # ge 
        if self.ge:
            self.target_ge_cnv_block = Sequential(
                nn.Conv1d(in_channels=1, out_channels=n_filters, kernel_size=8, stride=2),
                nn.ReLU(),
                nn.MaxPool1d(3),
                nn.Conv1d(in_channels=n_filters, out_channels=n_filters, kernel_size=8, stride=2),
                nn.ReLU(),
                nn.MaxPool1d(3),
                nn.Conv1d(in_channels=n_filters, out_channels=n_filters*2, kernel_size=4),
                nn.ReLU(),
                nn.MaxPool1d(3),
                nn.Conv1d(in_channels=n_filters*2, out_channels=n_filters*2, kernel_size=4),
                nn.ReLU(),
                nn.MaxPool1d(2),
                nn.Conv1d(in_channels=n_filters*2, out_channels=n_filters*4, kernel_size=4),
                nn.ReLU(),
                nn.MaxPool1d(2),
                nn.Conv1d(in_channels=n_filters*4, out_channels=n_filters*4, kernel_size=2),
                nn.ReLU(),
                nn.MaxPool1d(2),
                nn.Conv1d(in_channels=n_filters*4, out_channels=n_filters*2, kernel_size=2),
                nn.ReLU(),
                nn.MaxPool1d(2),
                nn.Flatten(),
                nn.Linear(512, output_dim),
                nn.ReLU()
            )
        # mut
        if self.mut:
            self.target_mut_cnv_block = Sequential(
                nn.Conv1d(in_channels=1, out_channels=n_filters, kernel_size=8),
                nn.ReLU(),
                nn.MaxPool1d(3),
                nn.Conv1d(in_channels=n_filters, out_channels=n_filters*2, kernel_size=8),
                nn.ReLU(),
                nn.MaxPool1d(3),
                nn.Conv1d(in_channels=n_filters*2, out_channels=n_filters*4, kernel_size=8),
                nn.ReLU(),
                nn.MaxPool1d(3),
                nn.Flatten(),
                nn.Linear(2944, output_dim),
                nn.ReLU(),
            )

        # meth
        if self.meth:
            self.target_meth_cnv_block = Sequential(
                nn.Conv1d(in_channels=1, out_channels=n_filters, kernel_size=8),
                nn.ReLU(),
                nn.MaxPool1d(3),
                nn.Conv1d(in_channels=n_filters, out_channels=n_filters*2, kernel_size=8),
                nn.ReLU(),
                nn.MaxPool1d(3),
                nn.Conv1d(in_channels=n_filters*2, out_channels=n_filters*4, kernel_size=8),
                nn.ReLU(),
                nn.MaxPool1d(3),
                nn.Flatten(),
                nn.Linear(1280, output_dim),
                nn.ReLU()
            )

        # synthetic omics data
        total = self.ge + self.mut + self.meth
        self.synthetic_omics = nn.Linear((total)*output_dim, 128)

        #attension
        self.key_xt = nn.Linear(output_dim, output_dim)
        self.key_drug = nn.Linear(output_dim, output_dim)

        self.at_fc = nn.Linear(3*output_dim, 1)

        # combined layers
        self.fc1 = nn.Linear(2*output_dim, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.out = nn.Linear(128, n_output)

        # activation and regularization
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, data):
        x_1_batch = data.x_1_batch
        x_1, edge_index_1,  = data.x_1, data.edge_index_1
        x_2_batch = data.x_2_batch
        x_2, edge_index_2,  = data.x_2, data.edge_index_2

    # drug 1
        x_1 = F.relu(self.conv1(x_1, edge_index_1))
        x_1 = self.bn1(x_1)
        x_1 = F.relu(self.conv2(x_1, edge_index_1))
        x_1 = self.bn2(x_1)
        x_1 = F.relu(self.conv3(x_1, edge_index_1))
        x_1 = self.bn3(x_1)
        x_1 = F.relu(self.conv4(x_1, edge_index_1))
        x_1 = self.bn4(x_1)
        x_1 = F.relu(self.conv5(x_1, edge_index_1))
        x_1 = self.bn5(x_1)
        x_1 = global_add_pool(x_1, x_1_batch)
        x_1 = F.relu(self.fc1_xd(x_1))
        x_1 = F.dropout(x_1, p=0.2, training=self.training)
    # drug 2
        x_2 = F.relu(self.conv1(x_2, edge_index_2))
        x_2 = self.bn1(x_2)
        x_2 = F.relu(self.conv2(x_2, edge_index_2))
        x_2 = self.bn2(x_2)
        x_2 = F.relu(self.conv3(x_2, edge_index_2))
        x_2 = self.bn3(x_2)
        x_2 = F.relu(self.conv4(x_2, edge_index_2))
        x_2 = self.bn4(x_2)
        x_2 = F.relu(self.conv5(x_2, edge_index_2))
        x_2 = self.bn5(x_2)
        x_2 = global_add_pool(x_2, x_2_batch)
        x_2 = F.relu(self.fc1_xd(x_2))
        x_2 = F.dropout(x_2, p=0.2, training=self.training)


        # protein input feed-forward:
        conv_xt_ge = torch.Tensor().to(device)
        conv_xt_mut = torch.Tensor().to(device)
        conv_xt_meth = torch.Tensor().to(device)

        if self.mut:
            target_mut = data.target_mut
            target_mut = target_mut[:,None,:]
            conv_xt_mut = self.target_mut_cnv_block(target_mut)

        if self.meth:
            target_meth = data.target_meth
            target_meth = target_meth[:,None,:]
            conv_xt_meth = self.target_meth_cnv_block(target_meth)

        if self.ge:
            target_ge = data.target_ge
            target_ge = target_ge[:,None,:]
            conv_xt_ge = self.target_ge_cnv_block(target_ge)
        # 1d conv layers

        xt = torch.cat((conv_xt_mut, conv_xt_meth, conv_xt_ge), 1)
        xt = self.synthetic_omics(xt)
        xt = self.relu(xt)

        key_xt = self.key_xt(xt)
        key_drug_1 = self.key_drug(x_1)
        key_drug_2 = self.key_drug(x_2)
        
        a_drug_1 = torch.exp(self.leaky_relu(self.at_fc(torch.cat((key_drug_1, key_xt, key_drug_2), 1))))
        a_drug_2 = torch.exp(self.leaky_relu(self.at_fc(torch.cat((key_drug_2, key_xt, key_drug_1), 1))))
        total = a_drug_1 + a_drug_2
        a_drug_1 = torch.div(a_drug_1, total)
        a_drug_2 = torch.div(a_drug_2, total)
        x_1 = a_drug_1*x_1
        x_2 = a_drug_2*x_2
        x_drug_combine = x_1 + x_2

        # concat
        xc = torch.cat((x_drug_combine, xt), 1)
        # add some dense layers
        xc = self.fc1(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        xc = self.fc2(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        out = self.out(xc)

        return out, a_drug_1, a_drug_2

In [5]:
def train(model, device, train_loader, optimizer, epoch, log_interval, critation):
    print('Training on {} samples...'.format(len(train_loader.dataset)))
    model.train()
    avg_loss = []
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        output, x_1, x_2 = model(data)
        loss = critation(output, data.y.view(-1, 1).float().to(device))
        loss.backward()
        optimizer.step()
        avg_loss.append(loss.item())
        text = 'Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,
                                                                        batch_idx * len(data.x_1),
                                                                        len(train_loader.dataset),
                                                                        100. * batch_idx / len(train_loader),
                                                                        loss.item())
        print(text)
    return sum(avg_loss)/len(avg_loss)

In [6]:
def predicting(model, device, loader, ats=False):
    model.eval()
    total_preds = torch.Tensor()
    total_labels = torch.Tensor()
    if ats:
        x_1_predicted = torch.Tensor()
        x_2_predicted = torch.Tensor()
        print('Make prediction for {} samples...'.format(len(loader.dataset)))
        with torch.no_grad():
            for data in loader:
                data = data.to(device)
                output, x_1, x_2 = model(data)
                total_preds = torch.cat((total_preds, output.cpu()), 0)
                total_labels = torch.cat((total_labels, data.y.view(-1, 1).cpu()), 0)
                x_1_predicted = torch.cat((x_1_predicted, x_1.cpu()), 0)
                x_2_predicted = torch.cat((x_2_predicted, x_2.cpu()), 0)
        return total_labels.numpy().flatten(),total_preds.numpy().flatten(), x_1_predicted, x_2_predicted
    else:
        print('Make prediction for {} samples...'.format(len(loader.dataset)))
        with torch.no_grad():
            for data in loader:
                data = data.to(device)
                output, x_1, x_2 = model(data)
                total_preds = torch.cat((total_preds, output.cpu()), 0)
                total_labels = torch.cat((total_labels, data.y.view(-1, 1).cpu()), 0)
        return total_labels.numpy().flatten(),total_preds.numpy().flatten()

In [7]:
def draw(list_, label, y_label, title):
    plt.figure()
    plt.plot(list_, label=label)
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel(y_label)
    plt.legend()
    # save image
    plt.savefig(title+".png")  # should before show method

In [ ]:
def return_ret(G, P):
    return [rmse(G,P),mse(G,P),pearson(G,P),spearman(G,P)]

def r2_rmse( g ):
            r2 =  mean_squared_error( g['synergy'], g['predict'] )
            count = len(g['synergy'])
            rmse = np.sqrt( mean_squared_error( g['synergy'], g['predict'] ) )
            return pd.Series( dict(  r2 = r2, rmse = rmse, count = count ) )
        
def get_top_data(r, df, top=10):
    G, P, x_1_ats, x_2_ats = r
    x_1_ats = np.array(x_1_ats)
    x_2_ats = np.array(x_2_ats)
    top = top*2
    abs_error = np.abs(G-P)
    index_top = abs_error.argsort()[:]
    values = abs_error[index_top]
    df_top = df.iloc[index_top].copy()
    df_top["log_synergy"] = G[index_top]
    df_top["predict"] = P[index_top]
    df_top["abs_error"] = values
    df_top["x_1_ats"] = x_1_ats[index_top]
    df_top["x_2_ats"] = x_2_ats[index_top]
    return df_top

# Load data

In [8]:
model_st = "GINConvNet"
dataset = 'GDSC'
train_batch = 1024
val_batch = 1024
test_batch = 1024
log_interval = 20

# Training

## Define paramters

In [12]:
lr = 0.001
num_epoch = 300
best_ret_test = None
print('Learning rate: ', lr)
print('Epochs: ', num_epoch)

Learning rate:  0.001
Epochs:  300


## Train model

In [ ]:
for data_type in data_types:
    for data_set in data_sets:
        
        data_path = f"data/split_data/{data_set}"
        data_processed_path = "data/split_data/{data_set}/processed/"
        model_st = "GINConvNet"
        dataset = 'GDSC'
        train_batch = 1024
        val_batch = 1024
        test_batch = 1024
        log_interval = 20

        print('\nrunning on ', model_st + '_' + dataset )
        train_data = TestbedDataset(root=data_path, dataset=dataset+"_"+'train_dc')
        val_data = TestbedDataset(root=data_path, dataset=dataset+"_"+'val_dc')
        test_data = TestbedDataset(root=data_path, dataset=dataset+"_"+'test_dc')
        # make data PyTorch
        # mini-batch processing ready
        train_loader = DataLoader(train_data, batch_size=train_batch, shuffle=True, follow_batch=['x_1', 'x_2'])
        val_loader = DataLoader(val_data, batch_size=val_batch, shuffle=False, follow_batch=['x_1', 'x_2'])
        test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False, follow_batch=['x_1', 'x_2'])

        print(device)
        modeling = GINConvNet(**data_type)
        model = modeling.to(device)

        lr = 0.001
        num_epoch = 300
        best_ret_test = None
        print('Learning rate: ', lr)
        print('Epochs: ', num_epoch)

        train_losses = []
        val_losses = []
        val_pearsons = []

        omics, name_omics = return_omics_type(data_type)
        save_path = "model/save_model/" + f"GIN_ADD_AT/{omics}/{name_omics}/{data_set}" + "/"
        print(save_path)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        best_mse = 1000
        best_pearson = 1
        best_epoch = -1

        best_val_losses = 10000

        ret_test_save = [1,1]

        model_file_name = save_path + 'best_model' + '.model'
        result_file_name = save_path + 'result_' + model_st + '_' + dataset +  '.csv'
        loss_fig_name = save_path + 'model_' + model_st + '_' + dataset + '_loss'
        pearson_fig_name = save_path + 'model_' + model_st + '_' + dataset + '_pearson'

        loss_fn = nn.MSELoss()
        for epoch in range(300):
            print('Epoch {} | Training on {} samples...'.format(epoch, len(train_loader.dataset)))
            avg_loss = []    
            train_loss = train(model, device, train_loader, optimizer, epoch+1, log_interval, loss_fn)
            #VAL:
            G,P = predicting(model, device, val_loader)
            ret = return_ret(G, P)

            train_losses.append(train_loss)
            val_losses.append(ret[1])
            val_pearsons.append(ret[2])

            if ret[1]<best_val_losses:
                best_val_losses = ret[1]
                G_test,P_test = predicting(model, device, test_loader)
                ret_test_save = return_ret(G_test, P_test)
                print("RMSE all test improved")
                torch.save(model.state_dict(), model_file_name)

            with open(result_file_name,'w') as f:
                f.write('ret_test: '+','.join(map(str,ret_test_save)) +"\n")

            draw_loss(train_losses, val_losses, loss_fig_name)
            draw_pearson(val_pearsons, pearson_fig_name)

            torch.save(model.state_dict(), model_file_name)
            log = [
                train_losses, val_losses
                ]

            with open(save_path+ "/log.pickle", "wb") as f:
                pickle.dump(log, f)

        data_split_path = f"data/split_data/{data_set}/"
        train_dc = pd.read_csv(data_split_path+"train_dc.csv")
        test_dc = pd.read_csv(data_split_path+"test_dc.csv")
        val_dc = pd.read_csv(data_split_path+"val_dc.csv")

        result_dict = {
            "test": (predicting(model, device, test_loader, ats=True), test_dc),
        }

        if os.path.exists(model_file_name):
            model.load_state_dict(torch.load(model_file_name))

        for key, value in tqdm(result_dict.items()):
            temp = get_top_data(value[0], value[1])
        temp.to_csv(save_path+"detail_result.csv", index=False)
        
        del model
        del train_data 
        del val_data 
        del test_data 
        torch.cuda.empty_cache() 
        gc.collect()